In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "LDL_Cholesterol_Levels"
cohort = "GSE28893"

# Input paths
in_trait_dir = "../../input/GEO/LDL_Cholesterol_Levels"
in_cohort_dir = "../../input/GEO/LDL_Cholesterol_Levels/GSE28893"

# Output paths
out_data_file = "../../output/preprocess/LDL_Cholesterol_Levels/GSE28893.csv"
out_gene_data_file = "../../output/preprocess/LDL_Cholesterol_Levels/gene_data/GSE28893.csv"
out_clinical_data_file = "../../output/preprocess/LDL_Cholesterol_Levels/clinical_data/GSE28893.csv"
json_path = "../../output/preprocess/LDL_Cholesterol_Levels/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genetic identification, replication, and functional fine-mapping of expression quantitative trait loci in primary human liver tissue [Illumina Expression Array]"
!Series_summary	"Most loci identified in genome wide association studies (GWAS) of complex traits reside in non-coding DNA and may contribute to phenotype via changes in gene regulation. The discovery of expression quantitative trait loci (?eQTLs?) can thus be used to more precisely identify modest but real disease associations and provide insights into their underlying molecular mechanisms. This is particularly true for analyses of expression in non-transformed cells from tissues relevant to the complex traits of interest. We have conducted two independent studies to identify genetic, including both SNPs and copy-number variants, and environmental determinants of human liver gene expression variation. We analyzed two sets of primary livers (primary dataset: n=220; replication dataset: n=

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import re
from typing import Optional, Dict, Any, Callable

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from liver tissue
# The series summary mentions "Illumina Expression Array" and gene expression quantification
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (LDL_Cholesterol_Levels)
# The background information mentions LDL cholesterol levels, but it's not directly in the sample characteristics
# Since the study is about eQTLs related to lipid levels including LDL cholesterol, this information might have been
# collected but not shown in the sample characteristics. In this case, we'll set trait_row to None.
trait_row = None

# For age
# Age data is available in row 1 of the sample characteristics dictionary
age_row = 1

# For gender
# Gender data is available in row 2 of the sample characteristics dictionary
gender_row = 2

# 2.2 Data Type Conversion
# Since trait data is not available, we'll define a placeholder function
def convert_trait(value):
    return None

# Age is a continuous variable
def convert_age(value):
    if value is None or not isinstance(value, str):
        return None
    
    # Extract the age value after colon
    match = re.search(r'age:\s*(\d+)', value)
    if match:
        return float(match.group(1))
    return None

# Gender is a binary variable (0 for female, 1 for male)
def convert_gender(value):
    if value is None or not isinstance(value, str):
        return None
    
    # Extract the gender value after colon
    match = re.search(r'gender:\s*(\w+)', value)
    if match:
        gender = match.group(1).upper()
        if gender == 'F':
            return 0
        elif gender == 'M':
            return 1
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None

# Initial filtering on dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep
if trait_row is not None:
    clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "clinical_data.csv"))
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Genetic identification, replication, and functional fine-mapping of expression quantitative trait loci in primary human liver tissue [Illumina Expression Array]"
Line 1: !Series_geo_accession	"GSE28893"
Line 2: !Series_status	"Public on Jun 27 2011"
Line 3: !Series_submission_date	"Apr 27 2011"
Line 4: !Series_last_update_date	"Aug 12 2019"
Line 5: !Series_pubmed_id	"21637794"
Line 6: !Series_pubmed_id	"23935528"
Line 7: !Series_summary	"Most loci identified in genome wide association studies (GWAS) of complex traits reside in non-coding DNA and may contribute to phenotype via changes in gene regulation. The discovery of expression quantitative trait loci (?eQTLs?) can thus be used to more precisely identify modest but real disease associations and provide insights into their underlying molecular mechanisms. This is particularly true for analyses of expression in non-transformed cells from tissues relevant to the complex traits 

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# The identifiers start with "ILMN_" which indicates they are Illumina probe IDs,
# not standard human gene symbols.
# Illumina probe IDs need to be mapped to standard gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1352865 rows

Gene annotation preview (first few rows):
{'ID': ['ILMN_1698220', 'ILMN_1810835', 'ILMN_1782944', 'ILMN_1692858', 'ILMN_1668162'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'RefSeq', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_13666', 'ILMN_10478', 'ILMN_27850', 'ILMN_10309', 'ILMN_7652'], 'Transcript': ['ILMN_13666', 'ILMN_175835', 'ILMN_27850', 'ILMN_10309', 'ILMN_7652'], 'ILMN_Gene': ['PHTF2', 'SPRR3', 'GPR37L1', 'FBXO25', 'DGAT2L3'], 'Source_Reference_ID': ['NM_020432.2', 'NM_005416.1', 'NM_004767.2', 'NM_012173.3', 'NM_001013579.1'], 'RefSeq_ID': ['NM_020432.2', 'NM_005416.1', 'NM_004767.2', 'NM_012173.3', 'NM_001013579.1'], 'Entrez_Gene_ID': [57157.0, 6707.0, 9283.0, 26260.0, 158833.0], 'GI': [40254932.0, 4885606.0, 31377792.0, 34878756.0, 61888901.0], 'Accession': ['NM_020432.2', 'NM_005416.1', 'NM_004767.2', 'NM_012173.3', 'NM_001013579.

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene identifiers and gene symbols in the annotation data
# From the gene annotation preview, we can see:
# - 'ID' column contains Illumina probe IDs (ILMN_*) which match our gene expression data identifiers
# - 'Symbol' column contains the standard human gene symbols we want to map to

# 2. Extract the gene mapping dataframe (probe ID to gene symbol)
prob_col = 'ID'
gene_col = 'Symbol'
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Created gene mapping with {len(gene_mapping)} entries")
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Convert probe-level measurements to gene-level expression data
# Apply the mapping to convert probe IDs to gene symbols
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Converted gene expression data from {len(gene_mapping)} probes to {len(gene_data)} unique genes")
print("Gene expression data preview (first 5 genes):")
print(preview_df(gene_data, 5))

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Created gene mapping with 22185 entries
Gene mapping preview:
{'ID': ['ILMN_1698220', 'ILMN_1810835', 'ILMN_1782944', 'ILMN_1692858', 'ILMN_1668162'], 'Gene': ['PHTF2', 'SPRR3', 'GPR37L1', 'FBXO25', 'DGAT2L3']}
Converted gene expression data from 22185 probes to 17609 unique genes
Gene expression data preview (first 5 genes):
{'GSM715884': [7.315778891, 21.197534268, 13.49944062, 7.188333454, 8.290600696], 'GSM715885': [7.699255452, 21.18170217, 13.9009021, 7.478674426, 7.541270113], 'GSM715886': [7.844632744, 21.328099792, 14.50270895, 7.425757607, 9.068905985], 'GSM715887': [7.232842539, 21.268624722, 11.76765243, 7.230454103, 7.323105565], 'GSM715888': [7.545822084, 20.945850837000002, 13.24399428, 7.400273045, 7.41528032], 'GSM715889': [7.539958013, 21.158119622, 13.10943042, 7.22065998, 7.334521849], 'GSM715890': [7.782385734, 21.109217142, 12.86917595, 7.208619332, 7.48532806], 'GSM715891': [7.531429413, 21.255664041, 13.7789095, 7.299108018, 7.834953184], 'GSM715892': [7.1060766

Gene expression data saved to ../../output/preprocess/LDL_Cholesterol_Levels/gene_data/GSE28893.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Load gene data that was saved in step 6
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data with {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
    
    # Normalize gene symbols using NCBI Gene database
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {gene_data.shape[0]} genes")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Clinical data analysis
# Based on the review of sample characteristics in Step 1, 
# we observed the trait (LDL_Cholesterol_Levels) is not directly available
print("\nAssessing clinical data availability...")
# From Step 2, we determined:
trait_row = None  # LDL_Cholesterol_Levels not directly available in sample characteristics
age_row = 1  # Age is in row 1
gender_row = 2  # Gender is in row 2

is_trait_available = trait_row is not None

# Skip clinical data extraction since trait is not available
if not is_trait_available:
    print("Required trait data (LDL_Cholesterol_Levels) is not available in the sample characteristics")
    selected_clinical_df = pd.DataFrame()
    linked_data = pd.DataFrame()  # Empty dataframe since we can't link without trait data
else:
    # This block wouldn't execute but is kept for completeness
    print("This dataset has trait information available")

# 3. Validate and save cohort information - final validation
print("\nPerforming final validation...")
note = "Dataset does not contain required trait information (LDL_Cholesterol_Levels)"

# Create empty DataFrame as required by the validation function
linked_data = pd.DataFrame()

# For datasets without trait data, is_biased should be set to True
# (as the absence of trait data makes it unusable for trait analysis)
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=True,  # Set to True when trait data is missing
    df=linked_data,
    note=note
)

# 4. Dataset is not usable for this trait, so we don't save linked data
print(f"\nDataset usability for {trait} association studies: {is_usable}")
print(f"Reason: {note}")


Normalizing gene symbols...
Loaded gene data with 17609 genes and 60 samples
After normalization: 16991 genes


Normalized gene data saved to ../../output/preprocess/LDL_Cholesterol_Levels/gene_data/GSE28893.csv

Assessing clinical data availability...
Required trait data (LDL_Cholesterol_Levels) is not available in the sample characteristics

Performing final validation...
Abnormality detected in the cohort: GSE28893. Preprocessing failed.

Dataset usability for LDL_Cholesterol_Levels association studies: False
Reason: Dataset does not contain required trait information (LDL_Cholesterol_Levels)
